#  Data Argument

In [1]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import os
import time

class VariationalEncoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalEncoder, self).__init__()
        self.linear1 = nn.Linear(63, 32)
        self.linear2 = nn.Linear(32, latent_dims)
        self.linear3 = nn.Linear(32, latent_dims)

        self.N = torch.distributions.Normal(0, 1)
        self.N.loc = self.N.loc#.cuda() # hack to get sampling on the GPU
        self.N.scale = self.N.scale#.cuda()
        self.kl = 0

    def forward(self, x):
        x = F.relu(self.linear1(x))
        mu =  self.linear2(x)
        sigma = torch.exp(self.linear3(x))
        z = mu + sigma*self.N.sample(mu.shape)
        self.kl = (sigma**2 + mu**2 - torch.log(sigma) - 1/2).sum()
        return z

class Decoder(nn.Module):
    def __init__(self, latent_dims):
        super(Decoder, self).__init__()
        self.linear1 = nn.Linear(latent_dims, 32)
        self.linear2 = nn.Linear(32, 63)

    def forward(self, z):
        z = F.relu(self.linear1(z))
        z = self.linear2(z)
        return z
    
class VariationalAutoencoder(nn.Module):
    def __init__(self, latent_dims):
        super(VariationalAutoencoder, self).__init__()
        self.encoder = VariationalEncoder(latent_dims)
        self.decoder = Decoder(latent_dims)

    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)
    def train(self, data, lr, epochs=200):
        opt = torch.optim.Adam(self.parameters(), lr = lr)
        loss_vae = []
        for epoch in range(epochs):
            for x in data:
                for sample in x:
                    sample = sample.to(device) # GPU
                    opt.zero_grad()
                    sample_hat = self.forward(sample)
                    loss = ((sample - sample_hat)**2).sum() + self.encoder.kl
                    loss.backward()
                    opt.step()
                    loss_vae.append(loss.item())
                #print( 'loss epoch %d: %f'% (epoch, loss.item()))
        return loss_vae
    def test(self, data):
        loss_vae = []
        for sample in data:
            sample_hat = self.forward(sample)
            loss = ((sample - sample_hat)**2).sum() + self.encoder.kl
            loss_vae.append(loss.item())
        return loss_vae
    
dataset = pd.DataFrame()
path_1 = "/kaggle/input/data-17-02-2023-60s"
dir_list_1 = os.listdir(path_1)
for name_file in dir_list_1:
    dataset = pd.concat([dataset,pd.read_csv(path_1 + "/" + name_file)])
    
path_2 = "/kaggle/input/test-data-21-02-2023"
dir_list_2 = os.listdir(path_2)
for name_file in dir_list_2:
    dataset = pd.concat([dataset,pd.read_csv(path_2 + "/" + name_file)])
    
dataset.fillna(0, inplace=True)
dataset = dataset.drop('EVENT_TIME', axis = 1)
dataset = dataset.drop('Unnamed: 0', axis = 1)
from sklearn.model_selection import train_test_split
train, test = train_test_split(dataset, test_size=0.02, shuffle=False)
features = train.columns
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)
train = pd.DataFrame(train, columns = features)
test = pd.DataFrame(test, columns = features)
torch.manual_seed(111)
batch_size = 32
train_loader = torch.utils.data.DataLoader(torch.Tensor(np.array(train)), batch_size=batch_size, shuffle=True)
device = ""
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
import time
start = time.time()
latent_dims = 16
vae = VariationalAutoencoder(latent_dims).to(device) # GPU
loss_train = vae.train(train_loader, 0.001, 10)

pd.DataFrame(loss_train).plot()
start = time.time()
loss_test = vae.test(torch.Tensor(np.array(test)))
pd.DataFrame(loss_test).plot()
end = time.time()
print(end - start)

# Link: 
https://avandekleut.github.io/vae/

# Evaluation model

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('bmh')
plt.xlabel('Sample')
plt.ylabel('Loss')
plt.plot(loss_train)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('bmh')
plt.xlabel('Sample')
plt.ylabel('Loss')
plt.plot(loss_test)

https://curiousily.com/posts/time-series-anomaly-detection-using-lstm-autoencoder-with-pytorch-in-python/